# Fine-tune LLMs to do Sarcasm interpretations

In [1]:
pip install nltk comet-ml emoji unbabel-comet datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.1/698.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/SarcasmNLP')

Mounted at /content/drive


In [103]:
# model_choice = 'gpt2'
model_choice = 'flan-t5-base'
# model_choice = 't5-base'
classifier_model_choice = 'bert-base-uncased'


In [63]:
mode = 'train'
# mode = 'evaluate'

## Load Model

### Classfication Model: bert-base-uncased

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification

if mode == 'train':
  classifier_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  classifier_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
else:
  classifier_tokenizer = BertTokenizer.from_pretrained(f'./results/{classifier_model_choice}/my_model')
  classifier_model = BertForSequenceClassification.from_pretrained(f'./results/{classifier_model_choice}/my_model')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Interpretation Models:

### GPT-2 small

In [84]:
if model_choice == 'gpt2':
  from transformers import GPT2Tokenizer, GPT2LMHeadModel
  if mode == 'train':
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
  else:
    tokenizer = GPT2Tokenizer.from_pretrained(f'./results/{model_choice}/my_model')
    model = GPT2LMHeadModel.from_pretrained(f'./results/{model_choice}/my_model')


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Google FLAN-T5-base

In [8]:
if model_choice == 'flan-t5-base':
  from transformers import T5Tokenizer, T5ForConditionalGeneration
  if mode == 'train':
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
  else:
    tokenizer = T5Tokenizer.from_pretrained(f'./results/{model_choice}/my_model')
    model = T5ForConditionalGeneration.from_pretrained(f'./results/{model_choice}/my_model')


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### T5-base

In [104]:
if model_choice == 't5-base':
  from transformers import T5Tokenizer, T5ForConditionalGeneration
  if mode == 'train':
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
  else:
    tokenizer = T5Tokenizer.from_pretrained(f'./results/{model_choice}/my_model')
    model = T5ForConditionalGeneration.from_pretrained(f'./results/{model_choice}/my_model')


## Load Data

In [105]:
import pandas as pd
def load_data():
  # if dataset == 'iSarcasm':
  #   return pd.read_csv('iSarcasm_pairs.tsv', sep='\t')
  # else:
  #   return pd.read_csv('GPT_pairs.tsv', sep='\t')
  dataset = pd.read_csv('combined_df.tsv', sep='\t')
  evaluation_dataset = pd.read_csv('iSarcasm_pairs_test.tsv', sep='\t')
  dataset['Translation'] = dataset['Translation'].fillna('')
  evaluation_dataset['Translation'] = evaluation_dataset['Translation'].fillna('')
  return dataset, evaluation_dataset

df, df_eval = load_data()

### Classification model:

### Intitialization

In [86]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [87]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [13]:
# Function to convert emojis to text, handling float values
import emoji

def convert_emojis(text):
    # Check if text is a float (potentially NaN) and convert to string
    if isinstance(text, float):
        text = str(text)
    return emoji.demojize(text, delimiters=(" ", " "))

# Apply emoji conversion to both input (sarcastic) and output (literal) text
train_df['Sarcastic'] = train_df['Sarcastic'].apply(convert_emojis)
valid_df['Sarcastic'] = valid_df['Sarcastic'].apply(convert_emojis)
test_df['Sarcastic'] = test_df['Sarcastic'].apply(convert_emojis)



In [14]:
def encode_texts(texts, targets=None, tokenizer=None):
    if targets is None:  # For single input (sarcasm detection)
        return classifier_tokenizer(
            texts.tolist(),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

In [15]:
import torch
from torch.utils.data import Dataset

# Custom dataset class for sarcasm classification
class SarcasmClassificationDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = encode_texts(texts.astype(str))
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.texts['input_ids'][idx],
            'attention_mask': self.texts['attention_mask'][idx],
            'labels': self.labels[idx]
        }

# Create datasets
train_dataset = SarcasmClassificationDataset(train_df['Sarcastic'], train_df['IsSarcastic'])
valid_dataset = SarcasmClassificationDataset(valid_df['Sarcastic'], valid_df['IsSarcastic'])
test_dataset = SarcasmClassificationDataset(test_df['Sarcastic'], test_df['IsSarcastic'])

### Training

In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    # Extract predictions and labels
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    # Return as dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [17]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results/{classifier_model_choice}',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_dir='./logs',
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    weight_decay=0.01,
    learning_rate=2e-5,
    save_strategy="epoch"
)

# Initialize Trainer for sarcasm classification
trainer = Trainer(
    model=classifier_model.to(device),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics       # Metrics function
)

In [18]:
if mode == 'train':
  # Train the sarcasm classifier
  trainer.train()
  # Save the model
  classifier_model.save_pretrained(f'./results/{classifier_model_choice}/my_model')
  classifier_tokenizer.save_pretrained(f'./results/{classifier_model_choice}/my_model')

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.368400,0.332631,0.855888,0.987755,0.753894,0.855124
2,0.294800,0.447265,0.869947,0.969582,0.794393,0.873288
3,0.188500,0.677581,0.843585,0.876623,0.841121,0.858506


In [19]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.33263063430786133, 'eval_accuracy': 0.8558875219683656, 'eval_precision': 0.9877551020408163, 'eval_recall': 0.7538940809968847, 'eval_f1': 0.8551236749116607, 'eval_runtime': 1.2569, 'eval_samples_per_second': 452.709, 'eval_steps_per_second': 57.285, 'epoch': 3.0}


### Interpretation Model:

### Initialization

In [106]:
df, df_eval = load_data()
df = df[df['IsSarcastic'] == 1]
print(df.shape)

(3130, 3)


In [107]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [108]:
add_prefix = lambda x: "Provide straightforward, literal translations for this sarcastic comment: " + str(x)

train_df['Input'] = train_df['Sarcastic'].apply(add_prefix)
valid_df['Input'] = valid_df['Sarcastic'].apply(add_prefix)
test_df['Input'] = test_df['Sarcastic'].apply(add_prefix)


In [109]:
train_df['Translation'] = train_df['Translation'].fillna("")
valid_df['Translation'] = valid_df['Translation'].fillna("")
test_df['Translation'] = test_df['Translation'].fillna("")

def tokenize_data(df):
    inputs = tokenizer(df['Input'].tolist(), padding=True, truncation=True, return_tensors="pt")
    targets = tokenizer(df['Translation'].tolist(), padding=True, truncation=True, return_tensors="pt")

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids'],
    }

# Tokenize train, validation, and test datasets
train_encodings = tokenize_data(train_df)
valid_encodings = tokenize_data(valid_df)
test_encodings = tokenize_data(test_df)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [110]:
import torch

class SarcasmTranslationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create datasets
train_dataset = SarcasmTranslationDataset(train_encodings)
valid_dataset = SarcasmTranslationDataset(valid_encodings)
test_dataset = SarcasmTranslationDataset(test_encodings)

In [111]:
import torch

class SarcasmTranslationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, sources):
        self.encodings = encodings
        self.sources = sources

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['source'] = self.sources[idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create datasets
train_dataset = SarcasmTranslationDataset(train_encodings, list(train_df['Sarcastic']))
valid_dataset = SarcasmTranslationDataset(valid_encodings, list(valid_df['Sarcastic']))
test_dataset = SarcasmTranslationDataset(test_encodings, list(test_df['Sarcastic']))

## Prepare Metrics

For colab, need to install additional packages (already in conda environment.yml)

In [26]:
import evaluate

# Load the metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
comet = evaluate.load("comet")  # Ensure COMET is installed and properly configured
chrf = evaluate.load("chrf")  # ChrF metric



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [27]:

def compute_metrics(decoded_preds, decoded_labels, sources):
    # # Get predictions and labels
    # predictions = pred.predictions[0]
    # labels = pred.label_ids
    # # if isinstance(predictions, list) and isinstance(predictions[0], list):
    # #     predictions = [pred[0] for pred in predictions]

    # # Decode predictions and labels
    # decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # ChrF
    chrf_result = chrf.compute(predictions=decoded_preds, references=decoded_labels)

    # ROUGE
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # COMET
    comet_result = comet.compute(predictions=decoded_preds, references=decoded_labels, sources=sources)



    # Combine the results, including all ROUGE scores
    metrics = {
        "bleu": bleu_result["bleu"],
        "chrf": chrf_result["score"],
        "comet": comet_result.get("mean_score", None),
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "rougeLsum": rouge_result.get("rougeLsum", None),
    }

    return metrics

## Training

In [28]:
model = model.to(device)

In [29]:
model.name_or_path

'google/flan-t5-base'

In [112]:
from transformers import Trainer, TrainingArguments

# Set training arguments
training_args = TrainingArguments(
    output_dir=f'./results/{model_choice}',
    eval_strategy="epoch",     # evaluation strategy to adopt during training
    learning_rate=2e-5,              # learning rate
    save_steps=10000,
    save_total_limit=1,              # keep only the most recent checkpoint
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=10,             # total number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    report_to="none",                # Disable wandb logging
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # compute_metrics=compute_metrics, # TODO change it as batch evaluation
)


In [31]:
if mode == 'train':
  trainer.train()
  # Save the model
  model.save_pretrained(f'./results/{model_choice}/my_model')
  tokenizer.save_pretrained(f'./results/{model_choice}/my_model')

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.282900,0.295788
2,0.205400,0.259190
3,0.212400,0.251486
4,0.207800,0.247312
5,0.216500,0.244657
6,0.191800,0.243103
7,0.167700,0.242125
8,0.182800,0.241445
9,0.195200,0.241118
10,0.175600,0.240953


## Evaluation

In [32]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.2409527748823166, 'eval_runtime': 1.9912, 'eval_samples_per_second': 157.19, 'eval_steps_per_second': 20.088, 'epoch': 10.0}


In [33]:
predictions, labels, _ = trainer.predict(test_dataset)

In [34]:
logits = torch.tensor(predictions[0])

# Select the token with the highest probability
predicted_token_ids = torch.argmax(logits, dim=-1)

# Decode the predicted tokens
decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

In [35]:
sources = [test_dataset[i]['source'] for i in range(len(test_dataset))]

In [36]:
metrics = compute_metrics(decoded_preds, decoded_labels, sources)
print(metrics)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'bleu': 0.1564890521796766, 'chrf': 40.95607149962518, 'comet': 0.5242897337808395, 'rouge1': 0.44794658034116386, 'rouge2': 0.206013327413476, 'rougeL': 0.4392334802618138, 'rougeLsum': 0.4391181705273445}


In [37]:
metrics = compute_metrics(sources, decoded_labels, sources)
print(metrics)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'bleu': 0.05456304383967352, 'chrf': 30.46879140238477, 'comet': 0.6295635588824178, 'rouge1': 0.23279939827351492, 'rouge2': 0.08526930014033529, 'rougeL': 0.20343531537645942, 'rougeLsum': 0.20301373500470155}


In [38]:
metrics = compute_metrics(decoded_preds, sources, sources)
print(metrics)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'bleu': 0.05189835307004795, 'chrf': 25.341788155862037, 'comet': 0.44055633622998247, 'rouge1': 0.2565136937000415, 'rouge2': 0.0888601690663639, 'rougeL': 0.230352525106181, 'rougeLsum': 0.2311064290720214}


In [39]:
# def inference(input_text):
#   if not input_text.startswith("Provide straightforward, literal translations for this sarcastic comment: "):
#     input_text = "Provide straightforward, literal translations for this sarcastic comment: " + input_text
#   # input_text = "Provide straightforward, literal translations for this sarcastic comment: I just absolutely LOVE how I've got to work outside for the next 3 days in the heatwave."

#   input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)
#   output_ids = model.generate(input_ids)
#   decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

#   return decoded_output

def classify_sarcasm(text):
    inputs = classifier_tokenizer(text, return_tensors="pt").to(device)  # Move inputs to the same device as the model
    # Move the model to the same device as the inputs
    classifier_model.to(device)
    outputs = classifier_model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction == 1

def generate_interpretation(input_text):
    if not input_text.startswith("Provide straightforward, literal translations for this sarcastic comment: "):
      input_text = "Provide straightforward, literal translations for this sarcastic comment: " + input_text
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)
    output_ids = model.generate(input_ids)
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return decoded_output


In [40]:
# Test on a few examples
evaluation_texts = df_eval.sample(n=20, random_state=42)
count = 0
for index, row in evaluation_texts.iterrows():
    is_sarcastic = classify_sarcasm(row['Sarcastic'])
    print(f"Evaluation {count}:")
    print(f"Original: {row['Sarcastic']}")
    print(f"Sarcastic: {'Yes' if is_sarcastic else 'No'}")
    if row['IsSarcastic'] == 1:
      interpretation = generate_interpretation(row['Sarcastic'])
      print(f"Interpretation: {interpretation}")
      print(f"Ground Truth: {row['Translation']}")
    print("-" * 20)
    count += 1

Evaluation 0:
Original: i’m dying
Sarcastic: No


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Interpretation: I'm dying
Ground Truth: i am so drained 
--------------------
Evaluation 1:
Original: Mental shoot out ends. Silent pause. Sirens start 🚨 Always so perfectly timed 🚔🚓👍🏽 #LineOfDuty #LineofDuty6 #LOD6 #LOD
Sarcastic: No
Interpretation: Mental shoot out ends. Silent pause. Sirens start  Always so perfectly
Ground Truth: The sirens always start just as the gunfire finishes 
--------------------
Evaluation 2:
Original: using loops, building an idea on that loop, and then deleting the loop has been a fun little exercise recently
Sarcastic: Yes
--------------------
Evaluation 3:
Original: "Alexa add small bananas to the shopping list." I've added smooth armours to your shopping list." Gee thanks Alexa! #alexa
Sarcastic: No
Interpretation: "Alexa add small bananas to the shopping list."
Ground Truth: i asked my smart speaker to add small bananas to the shopping list, but she misheard me.
--------------------
Evaluation 4:
Original: Just rediscovered that my nose is always in m

In [41]:
src = "Look at you, finishing all your snacks before dinner. What a healthy choice!"
truth = "Eating snacks before dinner is not a good decision for your health."
print(f" \nsrc: {src} \nisSarcastic: {classify_sarcasm(src)} \ntranslation: {generate_interpretation(src)} \nground_truth: {truth}")


 
src: Look at you, finishing all your snacks before dinner. What a healthy choice! 
isSarcastic: True 
translation: You often end up eating too much food before dinner. 
ground_truth: Eating snacks before dinner is not a good decision for your health.


### Evaluate on T5-base Model

In [113]:
model = model.to(device)

In [114]:
model.name_or_path

't5-base'

In [115]:
df2, eval_data = load_data()
df2['Input'] = df2['Sarcastic'].apply(add_prefix)

In [116]:
train_encodings = tokenize_data(df2)
gpt_pairs = SarcasmTranslationDataset(train_encodings, list(df2['Sarcastic']))

In [117]:
trainer_temp = Trainer(
    model=model,
    args=training_args,
    eval_dataset=gpt_pairs
    # compute_metrics=compute_metrics,
)

In [118]:
eval_results = trainer_temp.evaluate()
print(eval_results)

{'eval_loss': 25.63859748840332, 'eval_model_preparation_time': 0.0062, 'eval_runtime': 54.2005, 'eval_samples_per_second': 104.814, 'eval_steps_per_second': 13.118}


### Evaluate on GPT-4o-mini pairs

In [95]:
model = model.to(device)

In [96]:
model.name_or_path

'gpt2'

In [97]:
df2, eval_data = load_data()
df2['Input'] = df2['Sarcastic'].apply(add_prefix)

In [98]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class SarcasmTranslationDataset(Dataset):
    def __init__(self, encodings, labels, tokenizer, max_length=128):
        self.encodings = encodings
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        # Tokenize and process the labels similar to the input
        item['labels'] = item['input_ids'] # Assign input_ids as labels

        return item

    def __len__(self):
        return len(self.labels)

In [99]:
train_encodings = tokenize_data(df2)
gpt_pairs = SarcasmTranslationDataset(train_encodings, list(df2['Sarcastic']), tokenizer)

In [100]:
trainer_temp = Trainer(
    model=model,
    args=training_args,
    eval_dataset=gpt_pairs,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

<ipython-input-100-41ada6183bdc>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_temp = Trainer(


In [102]:
eval_results = trainer_temp.evaluate()
print(eval_results)

<ipython-input-98-51227b774467>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 9.566715240478516, 'eval_model_preparation_time': 0.0025, 'eval_runtime': 32.1057, 'eval_samples_per_second': 176.947, 'eval_steps_per_second': 22.146}
